In [1]:
import os
import pandas as pd
import pickle
import warnings

from tqdm.auto import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils.fixes import _joblib_parallel_args

from src.model.SupportSubsetEstimator import *
from src.model.moess import *
from src.utils import *


warnings.filterwarnings('ignore')

experiment = 'r2' 
# Experiment paths

print(f'Experiment: {experiment}\n')

results_folder = f'results/{experiment}'

os.makedirs(results_folder, exist_ok=True)

data = pd.read_parquet(f'../data/prep_real_data/{experiment}.parquet')

random_state = 1234

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(data.drop(columns=['y']))
y = data.y.values

# Calcular particiones
skf = StratifiedKFold(n_splits=10, random_state=random_state, shuffle=True)
ss_estimator = SupportSubsetEstimator()
ss_idx = []
train_idx = []
test_idx = []

for train_index, test_index in tqdm(list(skf.split(X, y)), desc="Estimating Support Subsets"):
    ss_estimator.fit(X[train_index], y[train_index])
    train_idx.append(train_index)
    test_idx.append(test_index)
    ss_idx.append(ss_estimator.supportsubset)

folds = list(zip(train_idx, test_idx, ss_idx))

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Experiment: r2



Estimating Support Subsets: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.01it/s]


In [2]:
# comprobar que entra bien el parámetro de active_set

class GridSearch_moess():

    def __init__(self,
                 method,
                 grid_params,
                 scoring=scaled_mcc,
                 folds=None,
                 n_jobs=-1,
                 random_state=1234,
                 kwargs=None,
                 verbose=True):

        self.method = method
        self.grid_params = grid_params
        self.scoring = scoring
        self.k_fold_splits = folds
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.kwargs = kwargs
        self.verbose = verbose


    def _eval(self, X, y, params):

        scores = []

        if self.kwargs is None:
            clf = self.method(**params)
        else:
            clf = self.method(**params, **self.kwargs)

        for train_index, test_index, ss_index in self.k_fold_splits:

            clf.fit(X[train_index], y[train_index], active_indexes=ss_index)

            scores.append(self.scoring(
                y[test_index], clf.predict(X[test_index])))

        return [clf, params, scores, np.mean(scores)]

    def fit(self, X, y):
        
        grid_params =list(ParameterGrid(self.grid_params))

        if self.verbose:
            with tqdm_joblib(tqdm(desc="Searching best hyperparameters", total=len(grid_params))) as progress_bar:
                self.scoring_results_ = Parallel(n_jobs=self.n_jobs, **_joblib_parallel_args(prefer='threads'))(
                    delayed(self._eval)(X, y, params)for params in grid_params)
        else:
            self.scoring_results_ = Parallel(n_jobs=self.n_jobs, **_joblib_parallel_args(prefer='threads'))(
                    delayed(self._eval)(X, y, params)for params in grid_params)

        self.best_index_ = np.argmax([result[3]
                                      for result in self.scoring_results_])

        self.best_estimator_ = self.scoring_results_[self.best_index_][0]
        self.best_params_ = self.scoring_results_[self.best_index_][1]
        self.best_score_ = np.mean(self.scoring_results_[self.best_index_][2]), np.std(
            self.scoring_results_[self.best_index_][2])

In [3]:
# hyperparameters gridsearch
random_state = 1234

grid_params = {
    'wrab': [True, False],
    'lam': [1, 3, 5],
    'prop_sample': [0.10, 0.20, 0.30],
    'n_learners': [10, 20, 30],
    'random_state': [random_state]
}

# MOESS-kNN 

kwargs = {'method':KNeighborsClassifier, 'params':{'n_neighbors' : list(range(1, 13, 2))}}

ensemble = MOESS

ensemble_grid = GridSearch_moess(ensemble, grid_params, scoring=scaled_mcc, folds=folds, n_jobs=-1, kwargs=kwargs)

print('MOESS knn: \n')

ensemble_grid.fit(X, y)

print(f'params: {ensemble_grid.best_params_}, score: {ensemble_grid.best_score_}\n')

MOESS knn: 



Searching best hyperparameters: 21654it [02:47, 129.44it/s]                                                                                                     

params: {'lam': 1, 'n_learners': 10, 'prop_sample': 0.1, 'random_state': 1234, 'wrab': False}, score: (0.7651230198161892, 0.07746415411216032)



In [6]:
ensemble_grid.scoring_results_

[[<src.model.moess.MOESS at 0x7fb250228070>,
  {'lam': 1,
   'n_learners': 10,
   'prop_sample': 0.1,
   'random_state': 1234,
   'wrab': True},
  [0.6666666666666666,
   0.7886751345948129,
   0.5487950036474266,
   0.5487950036474266,
   0.7988071523335984,
   0.6889822365046137,
   0.16458980337503154,
   0.7151657414559676,
   0.6443375672974064,
   0.5],
  0.606481430952295],
  {'lam': 1,
   'n_learners': 10,
   'prop_sample': 0.1,
   'random_state': 1234,
   'wrab': False},
  [0.753546276418555,
   0.8333333333333333,
   0.7988071523335984,
   0.6857142857142857,
   0.7988071523335984,
   0.9225771273642582,
   0.7333333333333334,
   0.7795084971874737,
   0.6304372986874878,
   0.7151657414559676],
  0.7651230198161892],
  {'lam': 1,
   'n_learners': 10,
   'prop_sample': 0.2,
   'random_state': 1234,
   'wrab': True},
  [0.753546276418555,
   0.7886751345948129,
   0.7439750182371333,
   0.6195228609334393,
   0.46220355269907726,
   0.7439750182371333,
   0.7795084971874737,
 

In [14]:
import pickle
with open('../results/r2/MOESS_dt.p', 'rb') as fin:
    moess_knn = pickle.load(fin)

In [15]:
moess_knn.best_score_

(0.7580706926757388, 0.08608961809515239)

In [20]:
with open('../results/r2/MOESS_svm.p', 'rb') as fin:
    moess_knn = pickle.load(fin)

In [21]:
moess_knn.best_score_

(0.7580706926757388, 0.08608961809515239)

In [1]:
from sklearn.utils.validation import check_random_state

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
random_instance = check_random_state(1234+2)

In [6]:
hot_set = range(150)
whole_set = range(150)
regular_set = list(set(whole_set)-set(hot_set_))

In [7]:
prop = round(random_instance.uniform(0, 1), 2)
hot_size = int(len(hot_set)*prop)
regular_size = int(len(regular_set)*(1-prop))

hot_idx = random_instance.choice(hot_set, hot_size)
regular_idx = random_instance.choice(regular_set, regular_size)

In [9]:
regular_set

[]

In [8]:
regular_idx

array([], dtype=float64)

In [16]:
data.values[regular_idx]

IndexError: arrays used as indices must be of integer (or boolean) type

In [44]:
random_instance.choice(regular_idx, regular_size)

ValueError: a must be 1-dimensional or an integer

In [14]:
import os
import pandas as pd
import pickle
import warnings

from tqdm.auto import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils.fixes import _joblib_parallel_args

from src.model.SupportSubsetEstimator import *
from src.model.moess import *
from src.utils import *


warnings.filterwarnings('ignore')

experiment = 'r2' 
# Experiment paths

print(f'Experiment: {experiment}\n')

results_folder = f'results/{experiment}'

os.makedirs(results_folder, exist_ok=True)

data = pd.read_parquet(f'../data/prep_real_data/{experiment}.parquet')

Experiment: r2



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [32]:
hot_size

78

In [34]:
regular_size

10

In [37]:
len(hot_idx)

78

In [38]:
len(regular_idx)

10

In [42]:
import numpy as np

In [43]:
np.hstack([hot_idx, regular_idx])

array([ 95,  79,  97,  55,  33,  73,  91,  55,  19,  86,  70,  60,  57,
        54,  28,  46,  95,  18,  92,  36,  63,  76,  73,  94,  83,  79,
        22,  31,  94,  32,  16,  35,  76,   6,  42,  57,  47,  15,  49,
        83,  40,  47,  24,  60,  67,  35,  65,  70,  36,  24,   8,  36,
        43,  39,  52,  82,  29,  71,  24,  21,  22,   9,  95,  53,  31,
        23,  58,  14,  89,  47,  58,  74,  37,  40,  73,  27,  20,  86,
       122, 130, 121, 121, 145, 145, 110, 105, 119, 144])